# Move amlr07-20221204 GCS files

The code for processiong amlr07-20221204 shadowgraph images (Cutter's code, adapted from Ohman et al methods) wrote out processed images into a single directory. The purpose of this notebook is to copy these files to their own folders, to be imported into VIAME-Web-AMLR.

Image types: 
- -ffPCG.png images: Flatfielded images, with Pixel Gamma Correction
- -imgff.png images: Flatfielded images
- .jpgorig-regions.jpg: Original jpg images, with red region bounding boxes pasted onto them

Note that both versions of flatfielded images have had other processing steps applied, such as masking.

Import modules, inlcuding 'sourcing' py file with functions

In [ ]:
from google.cloud import storage
from itertools import repeat
import subprocess
import pandas as pd
import multiprocessing as mp

%run -m file_move

## Variables and Prep

Set variable names, and generate list of files to rename

In [ ]:
storage_client = storage.Client(project = "ggn-nmfs-usamlr-dev-7b99")
source_bucket_name    = "amlr-imagery-proc-dev"
file_prefix    = "gliders/2022/amlr07-20221204/shadowgraph/images/Dir0079"

# file_substr    = "-ffPCG"
file_substr    = "-imgff"
# file_substr    = ".jpgorig-regions"

destination_bucket_name = "amlr-gliders-imagery-proc-dev"

In [ ]:
file_list_orig = list_blobs_with_prefix(
    source_bucket_name, file_prefix, file_substr=file_substr)    

In [ ]:
print(f"there are {len(file_list_orig)} files with {file_substr} " +
      f"with the prefix {source_bucket_name}/{file_prefix}")
print("First 10 files:") 
for i in file_list_orig[0:9]:
    print(i)
    
# # Summarize by string length, if desired
# lengths_list = (lambda x:[len(i) for i in x])(file_list_orig)
# pd.Series(lengths_list).value_counts()

Create new file list

In [ ]:
file_list_destination = []
for i in file_list_orig:
    i_orig = i
    i = i.replace("gliders/2022", "FREEBYRD/2023")
    i = i.replace("/shadowgraph/images", f"/images{file_substr}")
    i = i.replace("/output", "")
    file_list_destination.append(i)

In [ ]:
from random import sample
for i in sample(file_list_destination, 5):
    print(i)

## Copy
Copy the blobs, rather than rsyncing. Create the bucket object outside the functions to minimize the work that the functions need to do

In [ ]:
# source_bucket = storage_client.bucket(source_bucket_name)
# destination_bucket = storage_client.bucket(destination_bucket_name)

#### Testing cells

In [ ]:
# i = 1

# # source_blob = source_bucket.blob(file_list_orig[i])
# # blob_copy = source_bucket.copy_blob(
# #     source_blob, destination_bucket, file_list_destination[i]
# #     #if_generation_match=destination_generation_match_precondition,
# # )

# copy_blob_if_new(
#     source_bucket, file_list_orig[i], 
#     destination_bucket, file_list_destination[i]
# )


#### Run the full thing

Run the full thing in parallel

In [ ]:
%%time
numcores = 80 #mp.cpu_count()
print(f"Running with {numcores} cores")

with mp.Pool(numcores) as pool:
    out_list = pool.starmap(
        copy_blob_client_ifnew, 
        # copy_blob_if_new, 
        zip(repeat("ggn-nmfs-usamlr-dev-7b99"), 
            repeat(source_bucket_name), file_list_orig, 
            repeat(destination_bucket_name), file_list_destination)
    )

Run the full thing sequentially, so we can pass the client objects

In [ ]:
%%time
for (i, j) in zip(file_list_orig, file_list_destination):
    if not source_bucket.blob(i).exists():
        copy_blob(source_bucket, i, destination_bucket, j)

### Sanity checks

In [ ]:
# prefix_dest = "FREEBYRD/2023/amlr07-20221204/regions/Dir"
# file_list_dest = list_blobs_with_prefix(bucket_destination, prefix_dest)    
# print(f"there are {len(file_list_dest)} files " +
#       f"with the prefix {bucket_destination}/{prefix_dest}")

# lengths_list = (lambda x:[len(i) for i in x])(file_list_dest)
# pd.Series(lengths_list).value_counts()